##### Create dictionary of labels:urchan from Channels.csv

In [ ]:
import pandas as pd

# df = pd.DataFrame(pd.read_csv("E:\\Penn State Homework\\Homework\\CMPSC\\CMPSC 445 Proj\\Channels.csv"))

# channels_dict=df.set_index('labels')['urchan']
# channels_dict

labels
Fp1     1
Fp2     2
F3      3
F4      4
C3      5
       ..
PO8    63
Fpz    64
CPz    65
POz    66
Oz     67
Name: urchan, Length: 66, dtype: int64

In [4]:
import mne
import numpy as np
import torch
from sklearn.preprocessing import StandardScaler
from torch.utils.data import DataLoader, TensorDataset
import cupy
import os
# Load the CSV data
data_parkinsons = pd.read_csv('01_EyesOpen.csv', header=None)
data_non_parkinsons = pd.read_csv('02_EyesOpen.csv', header=None)

#MNE param
sfreq = 500.0  # Sampling frequency, adjust if known
n_channels = data_parkinsons.shape[1]
channel_names = [f"Ch{i+1}" for i in range(n_channels)]
# channel_names=list(channels_dict.keys())
info = mne.create_info(ch_names=channel_names, sfreq=sfreq, ch_types="eeg")

data_parkinsons = data_parkinsons.to_numpy().T  # (n_channels, n_times)
raw_parkinsons = mne.io.RawArray(data_parkinsons, info)

data_non_parkinsons = data_non_parkinsons.to_numpy().T
raw_non_parkinsons = mne.io.RawArray(data_non_parkinsons, info)

Creating RawArray with float64 data, n_channels=10, n_times=79081
    Range : 0 ... 79080 =      0.000 ...   158.160 secs
Ready.
Creating RawArray with float64 data, n_channels=10, n_times=76383
    Range : 0 ... 76382 =      0.000 ...   152.764 secs
Ready.


In [5]:
os.environ['MNE_USE_CUDA'] = 'true' 
mne.utils.set_config('MNE_USE_CUDA', 'true') 
raw_parkinsons.filter(1., 30.,n_jobs='cuda' ,method='fir',fir_design='firwin')
raw_non_parkinsons.filter(1., 30., n_jobs='cuda',method='fir',fir_design='firwin')

data_parkinsons_filtered = raw_parkinsons.get_data().T  # (n_samples, n_channels)
data_non_parkinsons_filtered = raw_non_parkinsons.get_data().T


Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 30 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Upper passband edge: 30.00 Hz
- Upper transition bandwidth: 7.50 Hz (-6 dB cutoff frequency: 33.75 Hz)
- Filter length: 1651 samples (3.302 s)

Now using CUDA device 0
Enabling CUDA with 10.83 GB available memory
Using CUDA for FFT FIR filtering
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 30 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge:

In [7]:
#Standardize
scaler = StandardScaler()
data_parkinsons_filtered = scaler.fit_transform(data_parkinsons_filtered)
data_non_parkinsons_filtered = scaler.transform(data_non_parkinsons_filtered)

#PyTorch tensors
X_parkinsons = torch.tensor(data_parkinsons_filtered, dtype=torch.float32)
X_non_parkinsons = torch.tensor(data_non_parkinsons_filtered, dtype=torch.float32)

#1=Parkinson's
#0=Non-Parkinson's
y_parkinsons = torch.ones(X_parkinsons.shape[0], dtype=torch.long)
y_non_parkinsons = torch.zeros(X_non_parkinsons.shape[0], dtype=torch.long)

#Combine datasets
X_combined = torch.cat((X_parkinsons, X_non_parkinsons), dim=0)
y_combined = torch.cat((y_parkinsons, y_non_parkinsons), dim=0)

#PyTorch Dataloader
dataset = TensorDataset(X_combined, y_combined)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)


In [9]:
import torch.nn as nn
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")
class EEGClassifier(nn.Module):
    def __init__(self):
        super(EEGClassifier, self).__init__()
        self.layer1= nn.Linear(10, 64)
        self.layer2 = nn.Linear(64, 32)
        self.layer3 = nn.Linear(32, 2)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.3)
    
    def forward(self, x):
        x = self.relu(self.layer1(x))
        x = self.dropout(x)
        x = self.relu(self.layer2(x))
        x = self.dropout(x)
        x = self.layer3(x)
        return x

model = EEGClassifier().to(device) 
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

# Training loop
num_epochs = 10
for epoch in range(num_epochs):
    for X_batch, y_batch in dataloader:
        X_batch = X_batch.to(device)
        y_batch = y_batch.to(device)
        optimizer.zero_grad()
        outputs = model(X_batch)
        loss = criterion(outputs, y_batch)
        loss.backward()
        optimizer.step()
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')


Using device: cuda
Epoch [1/10], Loss: 0.3666
Epoch [2/10], Loss: 0.3088
Epoch [3/10], Loss: 0.3395
Epoch [4/10], Loss: 0.1474
Epoch [5/10], Loss: 0.0852
Epoch [6/10], Loss: 0.1392
Epoch [7/10], Loss: 0.1883
Epoch [8/10], Loss: 0.0594
Epoch [9/10], Loss: 0.0195
Epoch [10/10], Loss: 0.2002
